# Parameter Identification Demo
## Levitador Magnético - Optimization Algorithms Comparison

This notebook demonstrates how to use the modular optimization framework to identify
parameters of the magnetic levitator system using various bio-inspired algorithms.

**Learning Objectives:**
- Load and configure the levitator benchmark problem
- Run different optimization algorithms
- Compare algorithm performance
- Visualize convergence and results

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path().absolute().parent))

from levitador_benchmark import LevitadorBenchmark
from src.optimization import (
    RandomSearch, DifferentialEvolution, GeneticAlgorithm,
    GreyWolfOptimizer, ArtificialBeeColony, HoneyBadgerAlgorithm
)
from src.visualization import plot_convergence, plot_boxplot

print("✓ Imports successful")

## 2. Load the Benchmark Problem

The levitador benchmark problem involves identifying three physical parameters:
- $k_0$: Base inductance [H]
- $k$: Inductance coefficient [H]
- $a$: Geometric parameter [m]

In [ ]:
# Create benchmark instance
problema = LevitadorBenchmark(
    datos_reales_path='../data/datos_levitador.txt',
    random_seed=42,
    verbose=True
)

print(f"\nProblem Dimensions: {problema.dim}")
print(f"Search Space: {problema.bounds}")
print(f"Variable Names: {problema.variable_names}")
print(f"Data Points: {len(problema.t_real)}")

## 3. Run Baseline: Random Search

Let's start with a simple baseline algorithm to understand the problem difficulty.

In [ ]:
# Random Search
rs = RandomSearch(problema, n_iterations=500, random_seed=42, verbose=False)
best_sol_rs, best_fit_rs = rs.optimize()

print("\n=== Random Search Results ===")
print(f"Best Solution: k0={best_sol_rs[0]:.6f}, k={best_sol_rs[1]:.6f}, a={best_sol_rs[2]:.6f}")
print(f"Best Fitness (MSE): {best_fit_rs:.6e}")
print(f"Function Evaluations: {rs.evaluations}")

## 4. Run Advanced Algorithms

Now let's try more sophisticated optimization algorithms.

In [ ]:
# Differential Evolution
print("\n[1/3] Running Differential Evolution...")
de = DifferentialEvolution(problema, pop_size=30, max_iter=100, 
                          F=0.8, CR=0.9, random_seed=42, verbose=False)
best_sol_de, best_fit_de = de.optimize()
print(f"  ✓ DE: MSE = {best_fit_de:.6e}")

# Genetic Algorithm
print("\n[2/3] Running Genetic Algorithm...")
ga = GeneticAlgorithm(problema, pop_size=30, generations=100,
                     random_seed=42, verbose=False)
best_sol_ga, best_fit_ga = ga.optimize()
print(f"  ✓ GA: MSE = {best_fit_ga:.6e}")

# Grey Wolf Optimizer
print("\n[3/3] Running Grey Wolf Optimizer...")
gwo = GreyWolfOptimizer(problema, pop_size=30, max_iter=100,
                       random_seed=42, verbose=False)
best_sol_gwo, best_fit_gwo = gwo.optimize()
print(f"  ✓ GWO: MSE = {best_fit_gwo:.6e}")

## 5. Compare Results

Let's create a summary table comparing all algorithms.

In [ ]:
import pandas as pd

# Create results dataframe
results_data = {
    'Algorithm': ['Random Search', 'Differential Evolution', 'Genetic Algorithm', 'Grey Wolf Optimizer'],
    'Best MSE': [best_fit_rs, best_fit_de, best_fit_ga, best_fit_gwo],
    'k0': [best_sol_rs[0], best_sol_de[0], best_sol_ga[0], best_sol_gwo[0]],
    'k': [best_sol_rs[1], best_sol_de[1], best_sol_ga[1], best_sol_gwo[1]],
    'a': [best_sol_rs[2], best_sol_de[2], best_sol_ga[2], best_sol_gwo[2]],
    'Evaluations': [rs.evaluations, de.evaluations, ga.evaluations, gwo.evaluations]
}

df = pd.DataFrame(results_data)
print("\n=== Algorithm Comparison ===")
print(df.to_string(index=False))

# Find best algorithm
best_idx = df['Best MSE'].idxmin()
print(f"\n🏆 Best Algorithm: {df.loc[best_idx, 'Algorithm']} with MSE = {df.loc[best_idx, 'Best MSE']:.6e}")

## 6. Visualize Convergence

Plot the convergence curves to see how each algorithm progressed.

In [ ]:
# Prepare convergence data
histories = {
    'Random Search': rs.get_convergence_curve(),
    'DE': de.get_convergence_curve(),
    'GA': ga.get_convergence_curve(),
    'GWO': gwo.get_convergence_curve()
}

# Plot
plt.figure(figsize=(12, 6))
for name, history in histories.items():
    plt.plot(history, label=name, linewidth=2, alpha=0.8)

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Best Fitness (MSE)', fontsize=12)
plt.title('Convergence Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.tight_layout()
plt.show()

print("\n✓ Convergence plot generated")

## 7. Validate Best Solution

Let's visualize how well the best solution fits the experimental data.

In [ ]:
# Get best solution
best_algorithm = df.loc[best_idx, 'Algorithm']
if best_algorithm == 'Differential Evolution':
    best_sol = best_sol_de
elif best_algorithm == 'Genetic Algorithm':
    best_sol = best_sol_ga
elif best_algorithm == 'Grey Wolf Optimizer':
    best_sol = best_sol_gwo
else:
    best_sol = best_sol_rs

# Visualize solution (if method exists)
try:
    problema.visualize_solution(best_sol)
    print("\n✓ Solution visualization generated")
except AttributeError:
    print("\n⚠ Visualization method not available in current benchmark version")
    print(f"Best solution: k0={best_sol[0]:.6f}, k={best_sol[1]:.6f}, a={best_sol[2]:.6f}")

## 8. Multiple Trials for Statistical Significance

Run multiple independent trials to get statistically significant results.

In [ ]:
def run_multiple_trials(algorithm_class, problema, n_trials=5, **kwargs):
    """Run multiple trials of an algorithm."""
    results = []
    for trial in range(n_trials):
        # Update seed for each trial
        if 'random_seed' in kwargs:
            kwargs['random_seed'] = kwargs['random_seed'] + trial
        
        optimizer = algorithm_class(problema, **kwargs)
        _, fitness = optimizer.optimize()
        results.append(fitness)
    
    return results

# Run 5 trials for each algorithm
print("Running 5 trials per algorithm...")
n_trials = 5

de_trials = run_multiple_trials(DifferentialEvolution, problema, n_trials,
                               pop_size=30, max_iter=50, F=0.8, CR=0.9,
                               random_seed=42, verbose=False)
print(f"  ✓ DE: Mean={np.mean(de_trials):.6e}, Std={np.std(de_trials):.6e}")

ga_trials = run_multiple_trials(GeneticAlgorithm, problema, n_trials,
                               pop_size=30, generations=50,
                               random_seed=42, verbose=False)
print(f"  ✓ GA: Mean={np.mean(ga_trials):.6e}, Std={np.std(ga_trials):.6e}")

gwo_trials = run_multiple_trials(GreyWolfOptimizer, problema, n_trials,
                                pop_size=30, max_iter=50,
                                random_seed=42, verbose=False)
print(f"  ✓ GWO: Mean={np.mean(gwo_trials):.6e}, Std={np.std(gwo_trials):.6e}")

## 9. Statistical Comparison with Boxplot

In [ ]:
# Create boxplot
trial_results = {
    'DE': de_trials,
    'GA': ga_trials,
    'GWO': gwo_trials
}

plt.figure(figsize=(10, 6))
plt.boxplot(trial_results.values(), labels=trial_results.keys(),
           patch_artist=True, notch=True, showmeans=True)
plt.ylabel('Best Fitness (MSE)', fontsize=12)
plt.title('Algorithm Performance Distribution (5 Trials)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')
plt.tight_layout()
plt.show()

print("\n✓ Boxplot generated")

## 10. Summary and Conclusions

**Key Findings:**

1. **Best Algorithm**: The algorithm that achieved the lowest MSE
2. **Convergence Speed**: How quickly each algorithm converges
3. **Consistency**: Standard deviation across trials indicates reliability
4. **Computational Cost**: Number of function evaluations needed

**Next Steps:**
- Try more algorithms (ABC, HBA, SOA, Tianji)
- Tune hyperparameters for better performance
- Run longer experiments with more iterations
- Use the CLI tool `scripts/run_benchmark.py` for comprehensive comparisons

## Appendix: Using Configuration Files

For production runs, use YAML configuration files:

```bash
# Quick test
python scripts/run_benchmark.py --config config/quick_test.yaml

# Full comparison
python scripts/run_benchmark.py --config config/full_comparison.yaml

# Custom selection
python scripts/run_benchmark.py --algorithms DE GA GWO --trials 10
```